In [7]:
import numpy as np
import pandas as pd
from PIL import Image
import collections, math

import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
#crop test images and cache to TEST_CROP_DIR by detections_full.pkl and RFCN/ImageSets/Main/test.txt
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

import pickle 
with open('../data/RFCN_detections/detections_full_resnet101_iter30000.pkl','rb') as f:
    detections_full = pickle.load(f, encoding='latin1') 
    
CONF_THRESH = 0.9
outputs = []
count = np.zeros(len(detections_full[0]))

for im in range(len(detections_full[0])):
#for im in range(1):
    outputs_im = []
    detects_im = []
    for cls in range(1,len(FISH_CLASSES)):
        detects_im_cls = detections_full[cls][im]
        for i in range(len(detects_im_cls)):
            if np.max(detects_im_cls[i,4+cls]) >= CONF_THRESH:
                outputs_im.append(detects_im_cls[i,:]) 
    count[im] = len(outputs_im)
    for cls in range(1,len(FISH_CLASSES)):  
        detects_im.append(detections_full[cls][im])
    detects_im = np.vstack(detects_im)
    if len(outputs_im) == 0:
        ind = np.argmax(np.max(detects_im[:,5:], axis=1))
        l = [0,0,0,0]
        l.extend(np.ndarray.tolist(detects_im[ind,4:]))
        outputs_im.append(l)
    outputs_im = np.asarray(outputs_im)
    outputs.append(outputs_im)

print(sum([outputs[i].shape[0] for i in range(len(outputs))]))

In [ ]:
collections.Counter(count9)

In [ ]:
np.where(count==2)

In [ ]:
count5 = count

In [ ]:
diff = [count9[i]==0 and count5[i]>1 for i in range(4777)]
ims_diff = []
for i in range(4777):
    if diff[i] == True:
        ims_diff.append(i)

ims_diff

In [ ]:
#visualize test image crop
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

with open("../RFCN/ImageSets/Main/test.txt","r") as f:
    ims = f.readlines()
test_files = [im[:-1]+'.jpg' for im in ims]

for j in np.ndarray.tolist(ims_2[0]):
    dets = outputs[j]
    print(dets)
    im = Image.open("../RFCN/JPEGImages/"+test_files[j])
    im = np.asarray(im)
    #im = im[:, :, (2, 1, 0)]
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(im, aspect='equal')
    for i in range(dets.shape[0]):
        bbox = dets[i, :4]
        score = np.amax(dets[i,4:])
        index = np.argmax(dets[i,4:])
        class_name = FISH_CLASSES[index]
        if not (bbox[0] == 0 and bbox[1] == 0 and bbox[2] == 0 and bbox[3] == 0):
            ax.add_patch(plt.Rectangle((bbox[0], bbox[1]),
                              bbox[2] - bbox[0],
                              bbox[3] - bbox[1], fill=False,
                              edgecolor='red', linewidth=3.5))
            ax.text(bbox[0], bbox[1] - 2,
                    '{:s} {:.3f}'.format(class_name, score),
                    bbox=dict(facecolor='blue', alpha=0.5),
                    fontsize=14, color='white')

    ax.set_title(('Image {} detections with '
                  'p({} | box) >= {:.1f}').format(j, class_name, CONF_THRESH),fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.draw()

In [8]:
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

import pickle 
with open('../data/RFCN_detections/detections_full_resnet101_iter30000.pkl','rb') as f:
    detections_full = pickle.load(f, encoding='latin1') 
    
CONF_THRESH = 0.9
outputs = np.ndarray((len(detections_full[0]), len(FISH_CLASSES)), dtype=np.float64)

for im in range(len(detections_full[0])):
#for im in range(1):
    outputs_im = []
    detects_im = []
    for cls in range(1,len(FISH_CLASSES)):
        detects_im_cls = detections_full[cls][im]
        for i in range(len(detects_im_cls)):
            if np.max(detects_im_cls[i,4+cls]) >= CONF_THRESH:
                outputs_im.append(detects_im_cls[i,:]) 
    count[im] = len(outputs_im)
    for cls in range(1,len(FISH_CLASSES)):  
        detects_im.append(detections_full[cls][im])
    detects_im = np.vstack(detects_im)
    if len(outputs_im) == 0:
        ind = np.argmax(np.max(detects_im[:,5:], axis=1))
        l = [0,0,0,0]
        l.extend(np.ndarray.tolist(detects_im[ind,4:]))
        outputs_im.append(l)
    outputs_im = np.asarray(outputs_im)
    outputs[im] = np.mean(outputs_im, axis=0)[4:]

CLIP_THRESH = 0.02
outputs_cliped = np.clip(outputs, CLIP_THRESH, 1, out=None)
outputs_cliped = outputs_cliped/np.sum(outputs_cliped, axis=1, keepdims=True)
    
test_preds = outputs_cliped[:1000]
train_preds = outputs_cliped[1000:]

with open("../RFCN/ImageSets/Main/test.txt","r") as f:
    ims = f.readlines()
test_files = [im[:-1]+'.jpg' for im in ims[:1000]]
train_files = [im[:-1] for im in ims[1000:]]
with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
    train_file_labels = f.readlines()

log_losses = []
for i in range(len(train_preds)):
    im = train_files[i]
    for im_label in train_file_labels:
        if im == im_label[:9]:
            label = im_label[10:-1]
            index = FISH_CLASSES.index(label)
            log_losses.append(-math.log(train_preds[i][index]))
log_loss = sum(log_losses) / float(len(log_losses))
print('logloss of train is', log_loss )

import datetime

submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
#submission.loc[:, 'image'] = pd.Series(test_files, index=submission.index)
submission.insert(0, 'image', test_files)

now = datetime.datetime.now()
info = 'RFCN_resnet101_conf{:.1f}_'.format(CONF_THRESH) + 'clip{:.2f}_'.format(CLIP_THRESH) + '{:.4f}'.format(log_loss)
sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
submission.to_csv(sub_file, index=False)

logloss of train is 0.14516134700848998
